In [ ]:
import timm
import torch
from tqdm import tqdm


import albumentations as A
from albumentations.pytorch import ToTensorV2

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
MODEL_PATH = ''

VALID_CSV_PATH = '/upstage-cv-classification-cv2/data/valid.csv'
VALID_IMAGE_PATH = '/upstage-cv-classification-cv2/data/valid'

# 모델 로드

In [ ]:
model = timm.create_model('efficientnet_b4', pretrained=False, num_classes=17).to(device)

# 모델 가중치 로드
model.load_state_dict(torch.load('efficientnet_b4_model.pth'))

# 모델 성능 확인

In [ ]:
# training config
img_size = 380
LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 32
num_workers = 0

patience = 5
min_delta = 0.001 # 성능 개선의 최소 변화량

# test image 변환
data_transform = A.Compose([
    A.Resize(height = img_size, width = img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
    ToTensorV2()
])

class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image = img)['image']
    
        return img, target

    def get_labels(self):
        return self.df[:, 1] 

In [ ]:
preds_list = []
targets_list = []

model.eval()

with torch.no_grad():
    pbar = tqdm(val_loader)
    for step, (image, targets) in enumerate(pbar):
        image = image.to(device)
        targets = targets.to(device)

        preds = model(image)
        print(f"step : {step}")
        print(f"preds : {preds}")
        loss = loss_fn(preds, targets)
    
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")